<a href="https://colab.research.google.com/github/Nitin286roxs/NLP/blob/main/lstm/fake_new_classifier_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
train_df=pd.read_csv("train.csv")

In [3]:
train_df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


# Null value check

In [4]:
train_df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [5]:
train_df.shape

(20800, 5)

# Drop Null value

In [8]:
train_df=train_df.dropna()

In [9]:
train_df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [20]:
X=train_df.drop(['label'], axis=1).reset_index()
X.shape

(18285, 5)

In [21]:
Y=train_df['label']
Y.shape

(18285,)

In [22]:
import tensorflow as tf
tf.__version__

'2.17.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [23]:
voc_size=5000

## One hot representation for news 'Title' of dataset

In [24]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Stemming

In [25]:
ps=PorterStemmer()
corpus_title=[]
documents=X['title']
for i in range(len(X)):
  review=re.sub('[^a-zA-z]', ' ', documents[i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
  review=" ".join(review)
  corpus_title.append(review)

In [27]:
corpus_title[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

## one hot encoding

In [28]:
one_hot_reps=[ one_hot(word, voc_size) for word in corpus_title]

In [29]:
one_hot_reps[0:5]

[[1638, 3572, 298, 2711, 3349, 3106, 3679, 186, 2028, 1092],
 [2843, 3267, 3500, 4761, 3605, 2474, 1662],
 [487, 3731, 1691, 878],
 [120, 3172, 4601, 873, 2570, 3170],
 [284, 3605, 2733, 2062, 4523, 2595, 3605, 3732, 3381, 962]]

## padding

In [30]:
sent_lenght=20
embedded_docs=pad_sequences(one_hot_reps, padding="pre", maxlen=sent_lenght)
embedded_docs[0:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1638,
        3572,  298, 2711, 3349, 3106, 3679,  186, 2028, 1092],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2843, 3267, 3500, 4761, 3605, 2474, 1662],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  487, 3731, 1691,  878],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,  120, 3172, 4601,  873, 2570, 3170],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  284,
        3605, 2733, 2062, 4523, 2595, 3605, 3732, 3381,  962]],
      dtype=int32)

## Create model

In [31]:
embedded_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedded_vector_features))
model.add(LSTM(100)) #100 Neaurons
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [34]:
len(embedded_docs), Y.shape

(18285, (18285,))

In [35]:
import numpy as np
X=np.array(embedded_docs)

## train test split

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3,random_state=42)

## train model

In [37]:
model.fit(X_train, Y_train,validation_data=(X_test, Y_test), epochs=10, batch_size=64)

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.8150 - loss: 0.4073 - val_accuracy: 0.9182 - val_loss: 0.1955
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.9452 - loss: 0.1414 - val_accuracy: 0.9183 - val_loss: 0.2139
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - accuracy: 0.9628 - loss: 0.1005 - val_accuracy: 0.9158 - val_loss: 0.2237
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 33ms/step - accuracy: 0.9725 - loss: 0.0789 - val_accuracy: 0.9125 - val_loss: 0.2451
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 9s 45ms/step - accuracy: 0.9826 - loss: 0.0559 - val_accuracy: 0.9078 - val_loss: 0.2616
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.9842 - loss: 0.0510 - val_accuracy: 0.9069 - val_loss: 0.3117
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 14s 49ms/step - accuracy: 0.9895 - loss: 0.0354 - val_accuracy: 0.9065 - val_loss: 0.3631
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 53ms/step - accuracy: 0.9920 - loss: 0.0267 - val

## prediction

In [38]:
Y_pred=model.predict(X_test)

172/172 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [39]:
Y_pred=np.where(Y_pred > 0.5, 1,0) ## AUC ROC curver

In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(Y_test, Y_pred)

array([[2821,  286],
       [ 262, 2117]])

## Accuracy score and classification report

In [43]:
from sklearn.metrics import accuracy_score, classification_report
score=accuracy_score(Y_pred, Y_test)
print(score)

0.9001093693036821


In [44]:
print(classification_report(Y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3083
           1       0.89      0.88      0.89      2403

    accuracy                           0.90      5486
   macro avg       0.90      0.90      0.90      5486
weighted avg       0.90      0.90      0.90      5486

